In [1]:
# linear regressions

In [2]:
import pandas as pd
import pickle
import pysal as ps
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy
import xlrd
from ukpostcodeutils import validation
from scipy.special import ndtri
from scipy.stats import chi2
from math import sqrt
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from tqdm import tqdm
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
from tqdm import tqdm

%matplotlib inline

/home/drcjar/anaconda3/envs/gds/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [3]:
infile = open('df','rb')
df = pickle.load(infile)
infile.close()

infile = open('dcut','rb')
dcut = pickle.load(infile)
infile.close()

infile = open('areacut','rb')
areacut = pickle.load(infile)
infile.close()

infile = open('rural_nsyc_vs_syc','rb')
rural_nsyc_vs_syc = pickle.load(infile)
infile.close()

infile = open('urban_nsyc_vs_syc','rb')
urban_nsyc_vs_syc = pickle.load(infile)
infile.close()

infile = open('rural_coastal_syc_vs_rural_coastal_nsyc','rb')
rural_coastal_syc_vs_rural_coastal_nsyc = pickle.load(infile)
infile.close()

infile = open('urban_coastal_syc_vs_rural_coastal_nsyc','rb')
urban_coastal_syc_vs_rural_coastal_nsyc = pickle.load(infile)
infile.close()


l = dict(zip(['cr', 'ar', 'ds', 'smr'], [i for i in range(0,4)]))
l2 = dict(zip(['cr2', 'ar2', 'ds2', 'smr2', 'da2'], [i for i in range(0,5)]))

# dcut is dockyard cuts
# areacut is area cuts
# cr = crude rate, ar = age-adjusted rate, ds = directly standardised rate, smr= indirect smr
# da = dockyard analysis

In [4]:
def olsmodel(df):
 #   X = df[['disttodock', 'disttopowerstn']]
    X = df['disttodock']
    y = df['directly_standardised_rate_per_1000'] * 1000 # rate now per million
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    print(model.summary())

In [5]:
olsmodel(pd.merge(areacut['men-only'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.088
Model:                                             OLS   Adj. R-squared:                  0.085
Method:                                  Least Squares   F-statistic:                     33.20
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           1.84e-08
Time:                                         00:25:13   Log-Likelihood:                -2231.3
No. Observations:                                  347   AIC:                             4467.
Df Residuals:                                      345   BIC:                             4474.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [6]:
olsmodel(pd.merge(areacut['men-only pre-2009'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.092
Model:                                             OLS   Adj. R-squared:                  0.090
Method:                                  Least Squares   F-statistic:                     35.05
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           7.75e-09
Time:                                         00:25:14   Log-Likelihood:                -2016.2
No. Observations:                                  347   AIC:                             4036.
Df Residuals:                                      345   BIC:                             4044.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [7]:
olsmodel(pd.merge(areacut['men-only post-2009'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.061
Model:                                             OLS   Adj. R-squared:                  0.058
Method:                                  Least Squares   F-statistic:                     22.30
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           3.40e-06
Time:                                         00:25:14   Log-Likelihood:                -2021.0
No. Observations:                                  347   AIC:                             4046.
Df Residuals:                                      345   BIC:                             4054.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [8]:
olsmodel(pd.merge(areacut['men-only pre-2009'][l['ds']]['WD16CD'], df.groupby('WD16CD').disttodock.mean().reset_index(), on='WD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.011
Model:                                             OLS   Adj. R-squared:                  0.011
Method:                                  Least Squares   F-statistic:                     81.33
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           2.42e-19
Time:                                         00:25:14   Log-Likelihood:                -48973.
No. Observations:                                 7181   AIC:                         9.795e+04
Df Residuals:                                     7179   BIC:                         9.796e+04
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [9]:
olsmodel(pd.merge(areacut['men-only post-2009'][l['ds']]['WD16CD'], df.groupby('WD16CD').disttodock.mean().reset_index(), on='WD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.006
Model:                                             OLS   Adj. R-squared:                  0.006
Method:                                  Least Squares   F-statistic:                     46.92
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           7.99e-12
Time:                                         00:25:14   Log-Likelihood:                -49386.
No. Observations:                                 7181   AIC:                         9.878e+04
Df Residuals:                                     7179   BIC:                         9.879e+04
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [10]:
olsmodel(pd.merge(areacut['men-only'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.088
Model:                                             OLS   Adj. R-squared:                  0.085
Method:                                  Least Squares   F-statistic:                     33.20
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           1.84e-08
Time:                                         00:25:14   Log-Likelihood:                -2231.3
No. Observations:                                  347   AIC:                             4467.
Df Residuals:                                      345   BIC:                             4474.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [11]:
olsmodel(pd.merge(areacut['men-only'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.088
Model:                                             OLS   Adj. R-squared:                  0.085
Method:                                  Least Squares   F-statistic:                     33.20
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           1.84e-08
Time:                                         00:25:14   Log-Likelihood:                -2231.3
No. Observations:                                  347   AIC:                             4467.
Df Residuals:                                      345   BIC:                             4474.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [12]:
olsmodel(pd.merge(areacut['men-only pre-2009'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.092
Model:                                             OLS   Adj. R-squared:                  0.090
Method:                                  Least Squares   F-statistic:                     35.05
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           7.75e-09
Time:                                         00:25:14   Log-Likelihood:                -2016.2
No. Observations:                                  347   AIC:                             4036.
Df Residuals:                                      345   BIC:                             4044.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [13]:
olsmodel(pd.merge(areacut['men-only post-2009'][l['ds']]['LAD16CD'], df.groupby('LAD16CD').disttodock.mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.061
Model:                                             OLS   Adj. R-squared:                  0.058
Method:                                  Least Squares   F-statistic:                     22.30
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           3.40e-06
Time:                                         00:25:14   Log-Likelihood:                -2021.0
No. Observations:                                  347   AIC:                             4046.
Df Residuals:                                      345   BIC:                             4054.
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [14]:
def olsmodel2(df):
 #   X = df[['disttodock', 'disttopowerstn']]
    X = df[['disttodock', 'imd', 'oacs']]
    y = df['directly_standardised_rate_per_1000']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    print(model.summary())

In [15]:
olsmodel2(pd.merge(areacut['all-years'][l['ds']]['WD16CD'], df[['WD16CD', 'imd', 'oacs', 'urban', 'disttodock']].groupby('WD16CD').mean().reset_index(), on='WD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.025
Model:                                             OLS   Adj. R-squared:                  0.024
Method:                                  Least Squares   F-statistic:                     60.89
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           7.24e-39
Time:                                         00:25:15   Log-Likelihood:                -2584.2
No. Observations:                                 7181   AIC:                             5176.
Df Residuals:                                     7177   BIC:                             5204.
Df Model:                                            3                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [16]:
olsmodel2(pd.merge(areacut['all-years'][l['ds']]['LAD16CD'], df[['LAD16CD', 'imd', 'oacs', 'urban', 'disttodock']].groupby('LAD16CD').mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.259
Model:                                             OLS   Adj. R-squared:                  0.252
Method:                                  Least Squares   F-statistic:                     39.92
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):           3.77e-22
Time:                                         00:25:15   Log-Likelihood:                 151.72
No. Observations:                                  347   AIC:                            -295.4
Df Residuals:                                      343   BIC:                            -280.0
Df Model:                                            3                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [17]:
def olsmodel3(df):
 #   X = df[['disttodock', 'disttopowerstn']]
    X = df['imd']
    y = df['directly_standardised_rate_per_1000']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)

    print(model.summary())

In [18]:
olsmodel3(pd.merge(areacut['all-years'][l['ds']]['LAD16CD'], df[['LAD16CD', 'imd']].groupby('LAD16CD').mean().reset_index(), on='LAD16CD'))

                                     OLS Regression Results                                    
Dep. Variable:     directly_standardised_rate_per_1000   R-squared:                       0.002
Model:                                             OLS   Adj. R-squared:                 -0.001
Method:                                  Least Squares   F-statistic:                    0.6327
Date:                                 Mon, 19 Aug 2019   Prob (F-statistic):              0.427
Time:                                         00:25:15   Log-Likelihood:                 100.08
No. Observations:                                  347   AIC:                            -196.2
Df Residuals:                                      345   BIC:                            -188.5
Df Model:                                            1                                         
Covariance Type:                             nonrobust                                         
                 coef    std err        

In [19]:
dcut['all-years'][l2['da2']]['LAD16CD']

,dock,population,percent_pop,observed,percent_deaths,crude_rate_per_1000,directly_standardised_rate_per_1000,dsr_lower,dsr_upper,expected,smr,lower_ci,upper_ci
0,0,46808832,84.744487,23566,81.410854,0.503452,0.503138,0.496727,0.509610,24546.212123,96.006666,94.784766,97.240384
1,1,8426421,15.255513,5381,18.589146,0.638587,0.640765,0.623890,0.657984,4400.787877,122.273560,119.028130,125.585060


In [20]:
dcut['pre-2009'][l2['da2']]['LAD16CD']

,dock,population,percent_pop,observed,percent_deaths,crude_rate_per_1000,directly_standardised_rate_per_1000,dsr_lower,dsr_upper,expected,smr,lower_ci,upper_ci
0,0,46808832,84.744487,10449,79.958678,0.223227,0.223089,0.218828,0.227413,11081.121149,94.295513,92.496060,96.121171
1,1,8426421,15.255513,2619,20.041322,0.310808,0.311791,0.300052,0.323875,1986.878851,131.814781,126.814311,136.961881


In [21]:
dcut['post-2009'][l2['da2']]['LAD16CD']

,dock,population,percent_pop,observed,percent_deaths,crude_rate_per_1000,directly_standardised_rate_per_1000,dsr_lower,dsr_upper,expected,smr,lower_ci,upper_ci
0,0,46808832,84.744487,13117,82.605958,0.280225,0.280049,0.275271,0.284888,13465.090974,97.414864,95.754829,99.096456
1,1,8426421,15.255513,2762,17.394042,0.327779,0.328974,0.316914,0.341378,2413.909026,114.420219,110.192417,118.768691


In [22]:
areacut['pre-2009'][l['ds']]['LAD16CD']['directly_standardised_rate_per_1000'].mean()

0.23254946917097027

In [23]:
areacut['post-2009'][l['ds']]['LAD16CD']['directly_standardised_rate_per_1000'].mean()

0.28620144153368154